In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_blobs
from IPython.display import display
import ast
import jenkspy

pd.options.display.max_columns = None

In [2]:
##########################################################################
#All of these functions are for initial csv processing
def ListCleaner(DirtyString):
    garbchars = "!?{}()[]/\\"
    
    DirtyString = DirtyString.lstrip("c")
    
    for char in garbchars:
        DirtyString = DirtyString.replace(char, "")
        
    DirtyString = "[" + DirtyString + "]"
    
    return DirtyString
        
    
def nastyfunction(row):
    try:
        return ast.literal_eval(row)
    except:
        return None
    
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])


def boolean_df(item_lists, unique_items):
    bool_dict = {}
    
    for i, item in enumerate(unique_items):
        
            bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    return pd.DataFrame(bool_dict)
##############################################################################   
def setCluster(recipeName="noinput"):
    if recipeName == "noinput":
        recipeName = mytempdataset.sample().iloc[0]["Name"]
        
    recipeindex = mytempdataset.index[mytempdataset["Name"] == recipeName].tolist()
    
    if len(recipeindex) == 0:
        print("Recipe Not found in list")
        return
    
    tmptruth = trutharray - trutharray[recipeindex]
    tmptruth = np.absolute(tmptruth)
    ingdiffs = np.sum(tmptruth, axis=1)
    mytempdataset["Ingredient Difference"] = ingdiffs
    breaks = jenkspy.jenks_breaks(mytempdataset["Ingredient Difference"], nb_class = 10)
    labels = ["c1","c2","c3","c4","c5","c6","c7","c8","c9","c10"]
    mytempdataset["Cluster"] = pd.cut(mytempdataset["Ingredient Difference"], bins=breaks, labels=labels)
    
    clusters = {}
    
    for i in labels:
        clusters[i] = {"nodes":mytempdataset[mytempdataset["Cluster"]==i],"weight":mytempdataset[mytempdataset["Cluster"]==i].shape[0]}
        
    return clusters

In [3]:
#Grab a subset of the data and drop columns we currently aren't interested in
mybigfatdataset = pd.read_csv("../../../Recipes/recipes.csv")
myfirstdataset = mybigfatdataset.sample(n=1000)
mytempdataset = myfirstdataset.copy(deep=True)
mytempdataset = mytempdataset[["Name","RecipeId","Description","RecipeIngredientParts"]]

In [4]:
#Clean the column of interest up, this is a disgusting method of cleaning and I don't respect it. But it works for now
tmping = mytempdataset["RecipeIngredientParts"].apply(lambda x: ListCleaner(x))
mytempdataset.update(tmping)
mytempdataset["RecipeIngredientParts"] = mytempdataset["RecipeIngredientParts"].apply(lambda x: nastyfunction(x))
mytempdataset = mytempdataset[mytempdataset["RecipeIngredientParts"].notnull()]
mytempdataset.reset_index(drop=True, inplace=True)

In [5]:
#Get a list of truth data that corresponds to each ingredient and whether or not a recipe uses it
IngredientsList = to_1D(mytempdataset["RecipeIngredientParts"]).value_counts()
ingbool = boolean_df(mytempdataset["RecipeIngredientParts"],IngredientsList.keys())
numpingbool = ingbool.to_numpy()
trutharray = numpingbool.astype(int)

*************************************************************************************

Everything above this markdown should only be ran once. It is all preliminary set up. Use setCluster() below to grab a dictionary of what you want. Input the recipe name to cluster around that recipes ingredient. Running with no input will choose a random recipe.

*************************************************************************************

In [6]:
tmp = setCluster()

In [7]:
print(tmp["c1"]["weight"])
print(tmp["c1"]["nodes"])

16
                                                  Name  RecipeId  \
50                             Mulberry and Cherry Jam    464630   
54                                          Butterbeer    239006   
146  Hershey's Hugs & Kisses Chocolate Pound Cake T...     42734   
163                       Rave Reviews Hot Fudge Sauce     16826   
184          Grilled Angel Food Cake With Strawberries    424488   
257                                  Basic Matzoh Brei    215465   
289                            Classic Hot Fudge Sauce    409483   
333                               Vanilla Almond Glaze    443319   
476                                   Nutmeg Meltaways     76868   
597                                  Brown Sugar Fudge    436810   
646                                    Chocolate Fudge    529623   
691                                  Fluffy Orange Pie    282020   
702                                             Kisses      6903   
789                           Carnation Milk 